# Main

In [1]:
import os
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

from data_preparation import prepare
from portfolio_model import generate_portfolio
from portfolio_model import analysis
from scipy.stats import linregress


absolute_path = os.getcwd()

# Refresh portfolio investments
input_file = prepare.load_inputs(absolute_path)
portfolio_investments_dict = prepare.load_portfolio_investments(absolute_path, input_file)

# Get benchmarks
model_start_date = datetime(1990, 1, 1)

benchmarks_ticker = {
    # 'SMEA.DE': 'MSCI_Europe_Small_Cap', # Ticker not working
    # 'XEMD.PA': 'MSCI_Europe_Mid_Cap', # Ticker not working
    'EXW1.DE': 'Eurostoxx_50',
    'EXSA.DE': 'Eurostoxx_600',
    'SWDA.L': 'MSCI_World',
    'FEZ': 'Eurostoxx_50',          # SPDR EURO STOXX 50 ETF (US)
    'IEUR': 'Eurostoxx_600',        # iShares Europe ETF (US)
    'URTH': 'MSCI_World'            # iShares MSCI World ETF (US)
}

benchmarks_dict = prepare.load_benchmarks(absolute_path, benchmarks_ticker, model_start_date)

# Aggregate portfolio investments to portfolio level
portfolio_df = generate_portfolio.aggregate_portfolio_investments(portfolio_investments_dict, absolute_path)

# Calculate portfolio alpha and beta against chosen benchmark
chosen_benchmark = 'EXSA.DE'

portfolio_alpha, portfolio_beta = analysis.calculate_portfolio_alpha_beta(portfolio_df, benchmarks_dict, chosen_benchmark)
print(f"Portfolio Alpha: {portfolio_alpha}, Portfolio Beta: {portfolio_beta}")



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Portfolio Alpha: 0.0006974509765691261, Portfolio Beta: 0.7458812438344264
